Execution for house price estimation

In [ ]:
# Import relevant files and libraries
import DataProcessor
import sys
import LRegression
import NLRegression
import performance_metrics
import Kmeans
import UserInterface
import numpy as np
import pandas as pd

Define the parameters



In [ ]:
# Allocate 80% of data to training
train_size = 0.8

# Identify features to be used
columnsToKeep = ['Price','Type','Bedroom','Bathroom','Car','Landsize','BuildingArea','Latitude','Longitude']
#columnsToKeep = ['Price', 'Bedroom', 'Bathroom']
#columnsToKeep = ['Price', 'Type', 'Landsize', 'BuildingArea']

# Path to dataset.
filepath = '../dataset/Melbourne_housing_FULL.csv'

# Identify prediction (ground truth)
prediction_column = 'Price'

Process raw data

In [ ]:
# Create a DataProcessor object and load the data
dp_obj = DataProcessor.DataProcessor()
dp_obj.LoadData(filepath)

# Remove incomplete points, remove unused features and shuffle the data
dp_obj.keepSelectedColumns(columnsToKeep)
dp_obj.filterMelbourneData()

dp_obj.remove_outliers(columnsToKeep, plot=False)

# dp_obj.encodeCategoricalData(['Type'])
dp_obj.shuffleData()
dp_obj.reduceDataSize(1500) # remove this to train on the full dataset


columnsToCorrelate = ['Price','Bedroom','Bathroom','Car','Landsize','BuildingArea','Latitude','Longitude']
dp_obj.calculateCorelation(columnsToCorrelate)

# Split the data into training and testing
train_X, train_Y, test_X, test_Y = dp_obj.splitData(train_size, prediction_column)

# print(train_X.head())

# Remove dwelling type and additional landsize / building area data for some processing
clean_columns = ['Type', 'Landsize','BuildingArea']
train_X_clean = train_X.drop(columns=clean_columns,axis=1)
test_X_clean = test_X.drop(columns=clean_columns,axis=1)

K-Means Clustering

In [ ]:
# Ashton
k = 3
# prefs = ['Type','Bedroom','Bathroom','Car','Landsize','BuildingArea','Latitude','Longitude']
prefs = ['Type','Bedroom','Bathroom','Car','Landsize','BuildingArea','Latitude','Longitude']

all_points = []
all_centroids = []

for pref in prefs:
    kmeansTrain = Kmeans.Kmeans(train_X, train_Y, k, pref)

    train_X, points, centroids = kmeansTrain.cluster()

    all_points.append(points)
    all_centroids.append(centroids)
    
    # Plot the data
    # kmeansTrain.plotKmean(train_X, points, centroids, pref)

for pref in prefs:
    kmeansTest = Kmeans.Kmeans(test_X, test_Y, k, pref)

    test_X, points, centroids = kmeansTest.cluster()

    all_points.append(points)
    all_centroids.append(centroids)

    # Plot the data
    # kmeansTest.plotKmean(test_X, points, centroids, pref)

# kmeans = Kmeans.Kmeans(train_X, train_Y, k, prefs)

# train_X, points, centroids = kmeans.cluster()

# all_points.append(points)
# all_centroids.append(centroids)

# kmeans.plotKmean(train_X, points, centroids, prefs)

print('train_X columns')
print(train_X.columns)

print('test_X columns')
print(test_X.columns)

Linear Regression

In [ ]:
# Train
lr = LRegression.LRegression()
values = ['Bedroom','Bathroom','Car','Landsize','BuildingArea','Latitude','Longitude']
values_h = ['Bedroom','Bathroom','Car','Landsize','Latitude','Longitude']
values_u = ['Bedroom','Bathroom','Car','BuildingArea','Latitude','Longitude']
assignments = ['AssignmentBedroom','AssignmentBathroom','AssignmentCar','AssignmentLandsize','AssignmentBuildingArea','AssignmentLatitude','AssignmentLongitude']

# For houses only
train_Xh = train_X[train_X['Type'] == 'h'].drop(['BuildingArea','AssignmentBuildingArea'],axis=1)
# For units only
train_Xu = train_X[train_X['Type'] == 'u'].drop(['Landsize','AssignmentLandsize'],axis=1)

# Initialise weights
wh = []
wu = []

# Initialise MSE
train_mse_h = []
train_mse_u = []

# Calculate weights for houses
for i in range(k):
    weight = lr.LinReg(train_Xh,train_Y[train_X['Type'] == 'h'],values,assignments,i)
    wh.append(weight)
wh = np.array(wh)
wh = wh.reshape(wh.shape[0],wh.shape[1],-1)

# Calculate MSE for houses
for i in range(k):
    train_mse_h.append(lr.MSE(train_Xh[values_h],wh[i,:],train_Y[train_X['Type'] == 'h']))
print("Training error (houses) = ", train_mse_h)

# Calculate weights for units
for i in range(k):
    weight = lr.LinReg(train_Xu,train_Y[train_X['Type'] == 'u'],values,assignments,i)
    wu.append(weight)
wu = np.array(wu)
wu = wu.reshape(wu.shape[0],wu.shape[1],-1)

# Calculate MSE for units
for i in range(k):
    train_mse_u.append(lr.MSE(train_Xu[values_u],wu[i,:],train_Y[train_X['Type'] == 'u']))
print("Training error (units) = ", train_mse_u)

In [ ]:
# Test

# For houses only
test_Xh = test_X[test_X['Type'] == 'h'].drop(['BuildingArea','AssignmentBuildingArea'],axis=1)
# For units only
test_Xu = test_X[test_X['Type'] == 'u'].drop(['Landsize','AssignmentLandsize'],axis=1)

# Initialise MSE
test_mse_h = []
test_mse_u = []

# Calculate MSE for houses
for i in range(k):
    test_mse_h.append(lr.MSE(test_Xh[values_h],wh[i,:],test_Y[test_X['Type'] == 'h']))
print("Testing error (houses) = ", test_mse_h)

# Calculate MSE for units
for i in range(k):
    test_mse_u.append(lr.MSE(test_Xu[values_u],wu[i,:],test_Y[test_X['Type'] == 'u']))
print("Testing error (units) = ", test_mse_u)

Non-Linear Regression (Gaussian Processing)

In [ ]:
# Train

# options: 'Type','Bedroom','Bathroom','Car','Size','Latitude','Longitude'

# ['Type', 'Bedroom', 'Bathroom', 'Car', 'Landsize', 'BuildingArea',
#        'Latitude', 'Longitude', 'Size', 'AssignmentType', 'AssignmentBedroom',
#        'AssignmentBathroom', 'AssignmentCar', 'AssignmentLandsize',
#        'AssignmentBuildingArea', 'AssignmentLatitude', 'AssignmentLongitude']

# ['Size','Latitude','Longitude']

featuresToTrain = ['Size','Latitude','Longitude', 'Bedroom', 'Bathroom'] # ['Size','Latitude', 'AssignmentLatitude' ,'Longitude', 'AssignmentLongitude']

# ask to train or load a model
query = input("Do you want to train a new model? (y/n): ")
if query == 'y':
    # Ask user for features to train on
    nlr = NLRegression.NLRegression(train_X, train_Y, test_X, test_Y, featuresToTrain)
    nlr.train()
    # ask to save
    query = input("Do you want to save the model? (y/n): ")
    if query == 'y':
        r_2, mae, mse = nlr.evaluate()
        filepath_ = '../saved_models/nlr_model_R_2_' + str(r_2)
        nlr.saveModel(filepath_)
else:
    nlr = NLRegression.NLRegression(train_X, train_Y, test_X, test_Y, featuresToTrain)
    filename_ = input("Enter the filename of the model to load: ")
    filepath_ = '../saved_models/' + filename_
    nlr.loadModel(filepath_)

# plotting predictions vs expected
nlr.plot()


In [ ]:
# slow but more accurate test
nlr.cross_validate()

In [ ]:
# Select a sample row and keep it as a Series
sample_number = 200

sample = test_X[featuresToTrain].iloc[sample_number]
print("sample: ", sample.values)

# Predict using the model with the scaled data
pred, bounds = nlr.predictActual(sample)
print("prediction: ", pred[0])
# print("bounds:     ", bounds[0], bounds[1])
print("actual:     ", test_Y.iloc[sample_number])

In [ ]:
# generate fake sample to test. must be pandas series
sample = pd.Series([100, 37.8, 144.9, 3, 2, 37.8, 144.9], index=featuresToTrain)
pred, bounds = nlr.predictActual(sample)
print("prediction: ", pred[0])
# print("bounds:     ", bounds[0], bounds[1])


Evaluation Metrics

In [ ]:
from performance_metrics import ElbowMethod
from DataProcessor import DataProcessor
import pandas as pd
from sklearn.impute import SimpleImputer

# Identify features to be used (excluding 'Type')
columnsToKeep = ['Price', 'Bedroom', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Latitude', 'Longitude']

# Path to dataset
filepath = '../dataset/Melbourne_housing_FULL.csv'

# Step 1: Load and preprocess the data
data_processor = DataProcessor()
data = data_processor.LoadData(filepath)

# Drop columns that are not relevant or cannot be converted to numeric
data = data_processor.keepSelectedColumns(columnsToKeep)

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Step 2: Use the ElbowMethod class with the preprocessed data
elbow_method = ElbowMethod(data)  # Initialize ElbowMethod
elbow_method.evaluate(max_clusters=10)
elbow_method.plot()
optimal_clusters = elbow_method.optimal_number_of_clusters()
print("Optimal number of clusters:", optimal_clusters)


In [ ]:
from performance_metrics import calculate_silhouette_score
# Assuming kmeans.cluster() returns x_new, points, centroids

kmeans = Kmeans.Kmeans(train_X, train_Y, k, pref)

x_new, points, centroids = kmeans.cluster()
assignment = x_new['Assignment' + pref].values  # Extract the cluster assignments

silhouette_avg = calculate_silhouette_score(points, assignment)
print("Silhouette Score:", silhouette_avg)

User input for preference

In [ ]:
# Calculate mean values if no user preference is provided
meanLand = dp_obj.getAverage('Landsize')
meanBuilding = dp_obj.getAverage('BuildingArea')

# Input user preferences
user_pref = UserInterface.UserInterface()
type, bedrooms, bathrooms, car, size, latitude, longitude = user_pref.Inputs(meanLand, meanBuilding)
print(type,bedrooms,bathrooms,car,size,latitude,longitude)

Final prediction

In [ ]:
# Return predicted value from best performing model
# @TODO: Update for building type
# @TODO: Update for 'luxury' level (cluster number)
# targets = [type,bedrooms,bathrooms,car,size,latitude,longitude]
targets = [3,2,1,1000,100,-37.5903,145.2507]
pred = lr.predict(targets,w[0,:])